# Домашнее задание № 7

## Задание 1 (4 балла) 

Обучите 2 модели похожую по архитектуре на модель из ULMFit для задачи классификации текста (датасет - lenta_40k )
В моделях должно быть как минимум два рекуррентных слоя, а финальный вектор для классификации составляться из последнего состояния RNN (так делалось в семинаре), а также AveragePooling и MaxPooling из всех векторов последовательности (конкатенируйте последнее состояния и результаты пулинга). В первой модели используйте обычные слои, а во второй Bidirectional. Рассчитайте по классовую точность/полноту/f-меру для каждой из модели (результаты не должны быть совсем близкие к нулю после обучения на хотя бы нескольких эпохах). 

> Так как мы работаем с тем же датасетом, что и в прошлом задании, я во многом буду переиспользовать тот код, что написал ранее. В частности, это относится к препроцессингу с лемматизацией и удалением стоп-слов, удалением двух последних классов из выборки и разделением на выборки.

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, classification_report
from collections import Counter
from tqdm.notebook import tqdm
import nltk
from nltk.corpus import stopwords
from pymystem3 import Mystem


nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/futyn/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

### 1. Загружаем и готовим данные

In [2]:
url = "https://github.com/mannefedov/compling_nlp_hse_course/raw/master/data/lenta_40k.csv.zip"
data = pd.read_csv(url)

data.dropna(subset=['topic', 'text'], inplace=True)

print(data['topic'].value_counts())

topic
Россия               9622
Мир                  8193
Экономика            4768
Спорт                3894
Наука и техника      3204
Культура             3183
Бывший СССР          3182
Интернет и СМИ       2643
Из жизни             1679
Дом                  1315
Силовые структуры    1203
Ценности              460
Бизнес                433
Путешествия           418
69-я параллель         82
Крым                   43
Культпросвет           25
Легпром                 6
Библиотека              3
Name: count, dtype: int64


In [3]:
# Убираем почти непредставленные классы
data = data[~data['topic'].isin(['Легпром', 'Библиотека'])]

### 2. Препроцессинг

Как и в прошлый раз — с лемматизацией и удалением стоп-слов, чтобы результат имел шанс быть получше.

In [4]:
mystem = Mystem()
russian_stopwords = set(stopwords.words("russian"))


def preprocess_text(text):
    lemmas = mystem.lemmatize(text)
    tokens = [token for token in lemmas if token.isalnum()
              and token not in russian_stopwords]
    return tokens

In [5]:
# Препроцессим весь датасет сразу
tqdm.pandas(desc="Preprocessing text")
data['processed_tokens'] = data['text'].progress_apply(preprocess_text)

Preprocessing text:   0%|          | 0/44347 [00:00<?, ?it/s]

### 3. Делаем словарь и датасет

In [6]:
vocab = Counter()
for tokens in data['processed_tokens']:
    vocab.update(tokens)

filtered_vocab = {word for word, count in vocab.items() if count > 15}

print(f"Original vocab size: {len(vocab)}")
print(f"Filtered vocab size: {len(filtered_vocab)}")

Original vocab size: 140210
Filtered vocab size: 20441


In [7]:
# Индексируем
word2id = {'PAD': 0, 'UNK': 1}
for word in filtered_vocab:
    word2id[word] = len(word2id)

# Маппим лейблы
id2label = {i: l for i, l in enumerate(set(data.topic))}
label2id = {l: i for i, l in id2label.items()}

texts_tokens = data['processed_tokens'].values
targets = [label2id[l] for l in data.topic]

In [8]:
# Делаем трейн и дев выборки
train_tokens, valid_tokens, train_targets, valid_targets = train_test_split(
    texts_tokens, targets, test_size=0.05, stratify=targets, random_state=42
)

In [9]:
# Датасет
class TextDataset(torch.utils.data.Dataset):
    def __init__(self, word2id, max_len, tokens_list, targets):
        self.texts = [torch.LongTensor(
            [word2id.get(w, 1) for w in t][:max_len]) for t in tokens_list]
        self.texts = torch.nn.utils.rnn.pad_sequence(
            self.texts, batch_first=True, padding_value=0)
        self.target = torch.LongTensor(targets)
        self.length = len(tokens_list)

    def __len__(self):
        return self.length

    def __getitem__(self, index):
        return self.texts[index], self.target[index]

In [10]:
MAX_LEN = 200
BATCH_SIZE = 256
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_dataset = TextDataset(word2id, MAX_LEN, train_tokens, train_targets)
valid_dataset = TextDataset(word2id, MAX_LEN, valid_tokens, valid_targets)

train_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size=BATCH_SIZE, shuffle=True)
valid_loader = torch.utils.data.DataLoader(
    valid_dataset, batch_size=BATCH_SIZE, shuffle=False)

### 4. Архитектура RNN

Мы определим один класс модели, который в зависимости от параметра направленности будет добавлять в нужные места либо однонаправленные, либо двунаправленные LSTM-слои. LSTM просто потому что оно из RNN кажется самым нормальным :) В линейных слоях классификации будем использовать батчнорм и дропаут.

In [11]:
class RNNULMFitClassifier(nn.Module):
    def __init__(
            self,
            vocab_size,
            emb_dim,
            hidden_size,
            output_dim,
            dropout=0.3,
            bidirectional=False):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, emb_dim, padding_idx=0)

        # Основной LSTM, 2 слоя
        self.rnn = nn.LSTM(
            emb_dim,
            hidden_size,
            num_layers=2,
            batch_first=True,
            dropout=dropout,
            bidirectional=bidirectional)

        # Множитель для размерности в зависимости от направленности
        self.direction_factor = 2 if bidirectional else 1

        # Размерность после конкатенации
        self.concat_dim = hidden_size * self.direction_factor * 3

        # Полносвязный слой с батчнормом и дропаутом
        self.fc1 = nn.Linear(self.concat_dim, 256)
        self.bn1 = nn.BatchNorm1d(256)
        self.dropout = nn.Dropout(dropout)
        self.fc2 = nn.Linear(256, output_dim)

    def forward(self, text):
        # Применяем эмбеддинги
        embedded = self.embedding(text)

        # Прогоняем через RNN
        output, (hidden, cell) = self.rnn(embedded)

        # Avg Pooling по всем скрытым состояниям
        avg_pool = torch.mean(output, dim=1)

        # Макспуллинг по всем скрытым состояниям
        max_pool, _ = torch.max(output, dim=1)

        # Последнее скрытое состояние
        last_hidden = output[:, -1, :]

        # Конкатенируем три вектора
        cat = torch.cat((last_hidden, avg_pool, max_pool), dim=1)

        # Классификация
        x = self.dropout(F.relu(self.bn1(self.fc1(cat))))
        return self.fc2(x)

### 5. Цикл обучения

In [12]:
def train_epoch(model, iterator, optimizer, criterion):
    model.train()
    metrics = {
        'loss': [],
        'precision': [],
        'recall': [],
        'f1': [],
        'accuracy': []}

    for texts, ys in tqdm(iterator, desc="Training", leave=False):
        texts, ys = texts.to(device), ys.to(device)
        optimizer.zero_grad()

        predictions = model(texts)
        loss = criterion(predictions, ys)
        loss.backward()
        optimizer.step()

        preds = predictions.argmax(1).cpu().numpy()
        y_true = ys.cpu().numpy()

        metrics['loss'].append(loss.item())
        metrics['precision'].append(
            precision_score(
                y_true,
                preds,
                average='macro',
                zero_division=0))
        metrics['recall'].append(
            recall_score(
                y_true,
                preds,
                average='macro',
                zero_division=0))
        metrics['f1'].append(
            f1_score(
                y_true,
                preds,
                average='macro',
                zero_division=0))
        metrics['accuracy'].append(accuracy_score(y_true, preds))

    return {k: np.mean(v) for k, v in metrics.items()}


def evaluate(model, iterator, criterion, print_report=False):
    model.eval()
    metrics = {
        'loss': [],
        'precision': [],
        'recall': [],
        'f1': [],
        'accuracy': []}
    all_preds = []
    all_targets = []

    with torch.no_grad():
        for texts, ys in iterator:
            texts, ys = texts.to(device), ys.to(device)
            predictions = model(texts)
            loss = criterion(predictions, ys)

            preds = predictions.argmax(1).cpu().numpy()
            y_true = ys.cpu().numpy()
            all_preds.extend(preds)
            all_targets.extend(y_true)

            metrics['loss'].append(loss.item())
            metrics['precision'].append(
                precision_score(
                    y_true,
                    preds,
                    average='macro',
                    zero_division=0))
            metrics['recall'].append(
                recall_score(
                    y_true,
                    preds,
                    average='macro',
                    zero_division=0))
            metrics['f1'].append(
                f1_score(
                    y_true,
                    preds,
                    average='macro',
                    zero_division=0))
            metrics['accuracy'].append(accuracy_score(y_true, preds))

    if print_report:
        print("\nClassification Report (Validation):")
        print(
            classification_report(
                all_targets,
                all_preds,
                target_names=[
                    id2label[i] for i in range(
                        len(id2label))]))

    return {k: np.mean(v) for k, v in metrics.items()}

### 6. Обучаем однонаправленную RNN

In [13]:
model_uni = RNNULMFitClassifier(len(word2id), emb_dim=100, hidden_size=128,
                                output_dim=len(label2id), bidirectional=False)
model_uni = model_uni.to(device)

# Оптимизатор и функция потерь
optimizer = optim.AdamW(model_uni.parameters(), lr=1e-3, weight_decay=1e-5)
criterion = nn.CrossEntropyLoss().to(device)

# Обучаем
num_epochs = 10
for epoch in range(num_epochs):
    print(f"Epoch {epoch+1}/{num_epochs}")
    train_metrics = train_epoch(model_uni, train_loader, optimizer, criterion)
    valid_metrics = evaluate(model_uni, valid_loader, criterion)

    print(
        f"Train - Loss: {train_metrics['loss']:.3f}, F1: {train_metrics['f1']:.3f}")
    print(
        f"Valid - Loss: {valid_metrics['loss']:.3f}, F1: {valid_metrics['f1']:.3f}")

Epoch 1/10


Training:   0%|          | 0/165 [00:00<?, ?it/s]

Train - Loss: 1.432, F1: 0.346
Valid - Loss: 1.060, F1: 0.467
Epoch 2/10


Training:   0%|          | 0/165 [00:00<?, ?it/s]

Train - Loss: 0.877, F1: 0.543
Valid - Loss: 0.975, F1: 0.536
Epoch 3/10


...

Train - Loss: 0.289, F1: 0.827
Valid - Loss: 0.847, F1: 0.655
Epoch 9/10


Training:   0%|          | 0/165 [00:00<?, ?it/s]

Train - Loss: 0.232, F1: 0.853
Valid - Loss: 0.888, F1: 0.652
Epoch 10/10


Training:   0%|          | 0/165 [00:00<?, ?it/s]

Train - Loss: 0.187, F1: 0.879
Valid - Loss: 0.889, F1: 0.646


In [14]:
# Оцениваем итоговую модель
evaluate(model_uni, valid_loader, criterion, print_report=True)


Classification Report (Validation):
                   precision    recall  f1-score   support

         Ценности       0.69      0.87      0.77        23
        Экономика       0.80      0.78      0.79       239
              Мир       0.73      0.83      0.78       410
            Спорт       0.94      0.97      0.95       195
           Бизнес       0.47      0.32      0.38        22
           Россия       0.78      0.73      0.75       481
              Дом       0.84      0.62      0.71        66
         Из жизни       0.56      0.61      0.58        84
      Путешествия       0.59      0.48      0.53        21
         Культура       0.84      0.79      0.82       159
    Культпросвет        0.00      0.00      0.00         1
   Интернет и СМИ       0.68      0.65      0.67       132
             Крым       0.00      0.00      0.00         2
Силовые структуры       0.57      0.52      0.54        60
   69-я параллель       0.00      0.00      0.00         4
  Наука и техника 

/home/futyn/miniconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/futyn/miniconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/futyn/miniconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


{'loss': np.float64(0.8893612093395658),
 'precision': np.float64(0.6617783932549006),
 'recall': np.float64(0.6515196762169871),
 'f1': np.float64(0.6460027906486592),
 'accuracy': np.float64(0.7664113562091504)}

### 7. Обучаем двунаправленную RNN

In [15]:
model_bi = RNNULMFitClassifier(len(word2id), emb_dim=100, hidden_size=128,
                               output_dim=len(label2id), bidirectional=True)
model_bi = model_bi.to(device)

# Оптимизатор и функция потерь
optimizer = optim.AdamW(model_bi.parameters(), lr=1e-3, weight_decay=1e-5)
criterion = nn.CrossEntropyLoss().to(device)

# Обучаем
num_epochs = 10
for epoch in range(num_epochs):
    print(f"Epoch {epoch+1}/{num_epochs}")
    train_metrics = train_epoch(model_bi, train_loader, optimizer, criterion)
    valid_metrics = evaluate(model_bi, valid_loader, criterion)

    print(
        f"Train - Loss: {train_metrics['loss']:.3f}, F1: {train_metrics['f1']:.3f}")
    print(
        f"Valid - Loss: {valid_metrics['loss']:.3f}, F1: {valid_metrics['f1']:.3f}")

Epoch 1/10


Training:   0%|          | 0/165 [00:00<?, ?it/s]

Train - Loss: 1.319, F1: 0.400
Valid - Loss: 0.947, F1: 0.516
Epoch 2/10


Training:   0%|          | 0/165 [00:00<?, ?it/s]

Train - Loss: 0.792, F1: 0.600
Valid - Loss: 0.839, F1: 0.597
Epoch 3/10


...

Train - Loss: 0.188, F1: 0.884
Valid - Loss: 0.859, F1: 0.654
Epoch 9/10


Training:   0%|          | 0/165 [00:00<?, ?it/s]

Train - Loss: 0.147, F1: 0.900
Valid - Loss: 0.931, F1: 0.660
Epoch 10/10


Training:   0%|          | 0/165 [00:00<?, ?it/s]

Train - Loss: 0.118, F1: 0.926
Valid - Loss: 0.955, F1: 0.687


In [16]:
# Оцениваем итоговую модель
evaluate(model_bi, valid_loader, criterion, print_report=True)


Classification Report (Validation):
                   precision    recall  f1-score   support

         Ценности       0.91      0.91      0.91        23
        Экономика       0.72      0.90      0.80       239
              Мир       0.81      0.76      0.78       410
            Спорт       0.95      0.94      0.95       195
           Бизнес       0.75      0.14      0.23        22
           Россия       0.76      0.77      0.77       481
              Дом       0.76      0.77      0.77        66
         Из жизни       0.55      0.62      0.58        84
      Путешествия       0.73      0.52      0.61        21
         Культура       0.79      0.90      0.84       159
    Культпросвет        0.00      0.00      0.00         1
   Интернет и СМИ       0.78      0.55      0.65       132
             Крым       0.00      0.00      0.00         2
Силовые структуры       0.62      0.47      0.53        60
   69-я параллель       1.00      0.50      0.67         4
  Наука и техника 

/home/futyn/miniconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/futyn/miniconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/futyn/miniconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


{'loss': np.float64(0.9547217620743645),
 'precision': np.float64(0.7039130603940047),
 'recall': np.float64(0.6905786513734706),
 'f1': np.float64(0.6867316981350401),
 'accuracy': np.float64(0.7802951388888889)}

Видно, что двунаправленная RNN в целом значительно выигрывает на этой задаче у однонаправленной. При этом обе модели существенно лучше справились, чем свёртки в предыдущем задании.

Двунаправленная модель обучается медленнее с точки зрения времени и, видимо, требует больше памяти, однако лучше справляется с дисбалансом классов (что видно по лучшему значению Macro F1 и, например, по лучшему предсказанию очень редкого класса "69-я параллель"), ну и быстрее приходит к лучшим метрикам с точки зрения количества эпох. Однако однонаправленная модель меньше переобучается и более стабильная и сбалансированная: precision и recall почти на одном уровне, тогда как двунаправленная модель более precision-biased, то есть более консервативна.

## Задание 2 (6 баллов)

На данных википедии (wikiann) обучите и сравните 3 модели:  
1) модель в которой как минимум два рекуррентных слоя, причем один из них GRU, а другой LSTM 
2) модель в которой как минимум 3 рекуррентных слоя идут друг за другом и при этом 2-ой и 3-й слои еще имеют residual connection к изначальным эмбедингам. Для того, чтобы сделать residual connection вам нужно будет использовать одинаковую размерность эмбедингов и количество unit'ов в RNN слоях, чтобы их можно было просуммировать 
3) модель в которой будут и рекуррентные и сверточные слои (как минимум 2 rnn и как минимум 2 cnn слоя). В cnn слоях будьте аккуратны с укорачиванием последовательности и используйте паддинг



Сравните качество по метрикам (точность/полнота/f-мера). Также придумайте несколько сложных примеров и проверьте, какие сущности определяет каждая из моделей.

In [2]:
from datasets import load_dataset
from huggingface_hub import login

login()

### 1. Загружаем данные и делаем словарь

In [3]:
dataset = load_dataset("wikiann", 'ru')

In [4]:
vocab = Counter()
for sent in dataset['train']['tokens']:
    vocab.update(sent)

# Немного почистим, оставляем слова, встретившиеся более 2 раз
filtered_vocab = {word for word, count in vocab.items() if count > 2}

word2id = {'PAD': 0, 'UNK': 1}
for word in filtered_vocab:
    word2id[word] = len(word2id)

# Теги из датасета (IOB)
id2label = {
    0: "O",
    1: "B-PER",
    2: "I-PER",
    3: "B-ORG",
    4: "I-ORG",
    5: "B-LOC",
    6: "I-LOC"}
label2id = {v: k for k, v in id2label.items()}

print(f"Vocab size: {len(word2id)}")
print(f"Labels: {label2id}")

Vocab size: 5445
Labels: {'O': 0, 'B-PER': 1, 'I-PER': 2, 'B-ORG': 3, 'I-ORG': 4, 'B-LOC': 5, 'I-LOC': 6}


### 2. Датасет с паддингом

In [5]:
class NERDataset(torch.utils.data.Dataset):
    def __init__(self, dataset_split, word2id, max_len=100):
        self.dataset = dataset_split
        self.word2id = word2id
        self.max_len = max_len

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, index):
        tokens = self.dataset[index]['tokens']
        tags = self.dataset[index]['ner_tags']

        # Конвертируем токены в индексы
        ids = [self.word2id.get(t, 1) for t in tokens][:self.max_len]
        # Теги тоже обрезаем
        labels = tags[:self.max_len]

        # Паддинг
        pad_len = self.max_len - len(ids)
        if pad_len > 0:
            ids = ids + [0] * pad_len
            # Используем -100 для паддинга меток, чтобы loss их игнорировал
            labels = labels + [-100] * pad_len

        return torch.LongTensor(ids), torch.LongTensor(labels)

In [6]:
MAX_LEN = 64
BATCH_SIZE = 256

train_data = NERDataset(dataset['train'], word2id, MAX_LEN)
valid_data = NERDataset(
    dataset['validation'],
    word2id,
    MAX_LEN)  # Validation split
test_data = NERDataset(dataset['test'], word2id, MAX_LEN)

train_loader = torch.utils.data.DataLoader(
    train_data, batch_size=BATCH_SIZE, shuffle=True)
valid_loader = torch.utils.data.DataLoader(
    valid_data, batch_size=BATCH_SIZE, shuffle=False)
test_loader = torch.utils.data.DataLoader(
    test_data, batch_size=BATCH_SIZE, shuffle=False)

### 3. Модифицированный для NER цикл обучения

В основном изменения касаются подсчёта метрик, так как теперь мы предсказываем метку для каждого слова

In [7]:
def train_ner_epoch(model, iterator, optimizer, criterion):
    model.train()
    metrics = {
        'loss': [],
        'precision': [],
        'recall': [],
        'f1': [],
        'accuracy': []}

    for texts, labels in tqdm(iterator, desc="Training", leave=False):
        texts, labels = texts.to(device), labels.to(device)
        optimizer.zero_grad()

        predictions = model(texts)  # [Batch, Seq, Classes]

        # Переставляем размерности для CrossEntropyLoss
        predictions_flat = predictions.view(-1, predictions.shape[-1])
        labels_flat = labels.view(-1)

        loss = criterion(predictions_flat, labels_flat)
        loss.backward()
        optimizer.step()

        # Метрики считаем без паддинга
        mask = labels_flat != -100
        preds = predictions_flat.argmax(1)[mask].cpu().numpy()
        y_true = labels_flat[mask].cpu().numpy()

        metrics['loss'].append(loss.item())
        metrics['precision'].append(
            precision_score(
                y_true,
                preds,
                average='macro',
                zero_division=0))
        metrics['recall'].append(
            recall_score(
                y_true,
                preds,
                average='macro',
                zero_division=0))
        metrics['f1'].append(
            f1_score(
                y_true,
                preds,
                average='macro',
                zero_division=0))
        metrics['accuracy'].append(accuracy_score(y_true, preds))

    return {k: np.mean(v) for k, v in metrics.items()}


def evaluate_ner(model, iterator, criterion):
    model.eval()
    metrics = {
        'loss': [],
        'precision': [],
        'recall': [],
        'f1': [],
        'accuracy': []}

    with torch.no_grad():
        for texts, labels in iterator:
            texts, labels = texts.to(device), labels.to(device)
            predictions = model(texts)

            predictions_flat = predictions.view(-1, predictions.shape[-1])
            labels_flat = labels.view(-1)

            loss = criterion(predictions_flat, labels_flat)

            mask = labels_flat != -100
            preds = predictions_flat.argmax(1)[mask].cpu().numpy()
            y_true = labels_flat[mask].cpu().numpy()

            metrics['loss'].append(loss.item())
            metrics['precision'].append(
                precision_score(
                    y_true,
                    preds,
                    average='macro',
                    zero_division=0))
            metrics['recall'].append(
                recall_score(
                    y_true,
                    preds,
                    average='macro',
                    zero_division=0))
            metrics['f1'].append(
                f1_score(
                    y_true,
                    preds,
                    average='macro',
                    zero_division=0))
            metrics['accuracy'].append(accuracy_score(y_true, preds))

    return {k: np.mean(v) for k, v in metrics.items()}

### 4. Модель 1: GRU + LSTM

In [8]:
class HybridRNN(nn.Module):
    def __init__(
            self,
            vocab_size,
            emb_dim,
            hidden_size,
            output_dim,
            dropout=0.3):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, emb_dim, padding_idx=0)
        self.dropout = nn.Dropout(dropout)

        # 1 слой - двунаправленный GRU
        self.gru = nn.GRU(
            emb_dim,
            hidden_size,
            batch_first=True,
            bidirectional=True)

        # 2 слой - двунаправленный LSTM
        self.lstm = nn.LSTM(
            hidden_size * 2,
            hidden_size,
            batch_first=True,
            bidirectional=True)

        # Полносвязный слой
        self.fc = nn.Linear(hidden_size * 2, output_dim)

    def forward(self, text):
        embedded = self.dropout(self.embedding(text))

        # Проход через GRU
        gru_out, _ = self.gru(embedded)

        # Проход через LSTM
        lstm_out, _ = self.lstm(gru_out)

        # Предсказание для каждого токена
        return self.fc(self.dropout(lstm_out))

### 5. Модель 2: Residual RNN

In [9]:
class ResidualRNN(nn.Module):
    def __init__(self, vocab_size, hidden_size, output_dim, dropout=0.3):
        super().__init__()
        # Размер эмбеддинга равен размеру скрытого слоя для сложения
        self.embedding = nn.Embedding(vocab_size, hidden_size, padding_idx=0)
        self.dropout = nn.Dropout(dropout)

        # 3 последовательных однонаправленных слоя
        self.rnn1 = nn.LSTM(hidden_size, hidden_size, batch_first=True)
        self.rnn2 = nn.LSTM(hidden_size, hidden_size, batch_first=True)
        self.rnn3 = nn.LSTM(hidden_size, hidden_size, batch_first=True)

        self.fc = nn.Linear(hidden_size, output_dim)

    def forward(self, text):
        # Исходные эмбеддинги
        initial_emb = self.dropout(self.embedding(text))

        # 1 слой: обычный проход
        out1, _ = self.rnn1(initial_emb)

        # 2 слой: вход + исходный эмбеддинг (Residual connection)
        res_input2 = out1 + initial_emb
        out2, _ = self.rnn2(res_input2)

        # 3 слой: вход + исходный эмбеддинг
        res_input3 = out2 + initial_emb
        out3, _ = self.rnn3(res_input3)

        return self.fc(self.dropout(out3))

### 6. Модель 3: CNN + RNN

In [10]:
class CNNRNNModel(nn.Module):
    def __init__(
            self,
            vocab_size,
            emb_dim,
            hidden_size,
            output_dim,
            dropout=0.3):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, emb_dim, padding_idx=0)
        self.dropout = nn.Dropout(dropout)

        # CNN слои
        self.conv1 = nn.Conv1d(emb_dim, hidden_size, kernel_size=3, padding=1)
        self.conv2 = nn.Conv1d(
            hidden_size,
            hidden_size,
            kernel_size=3,
            padding=1)

        # RNN слои
        self.rnn1 = nn.LSTM(
            hidden_size,
            hidden_size,
            batch_first=True,
            bidirectional=True)
        self.rnn2 = nn.LSTM(
            hidden_size * 2,
            hidden_size,
            batch_first=True,
            bidirectional=True)

        self.fc = nn.Linear(hidden_size * 2, output_dim)

    def forward(self, text):
        embedded = self.dropout(self.embedding(text))

        x = embedded.permute(0, 2, 1)

        # CNN блок
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))

        # Возвращаем размерность для RNN
        x = x.permute(0, 2, 1)

        # RNN блок
        x, _ = self.rnn1(x)
        x, _ = self.rnn2(x)

        return self.fc(self.dropout(x))

### 7. Обучение всех моделей

In [11]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
criterion = nn.CrossEntropyLoss(ignore_index=-100)
EPOCHS = 10

In [12]:
models_to_train = {
    "GRU+LSTM": HybridRNN(len(word2id), 128, 128, len(label2id)),
    "Residual RNN": ResidualRNN(len(word2id), 128, len(label2id)),
    "CNN + RNN": CNNRNNModel(len(word2id), 128, 128, len(label2id))
}

for name, model in models_to_train.items():
    print(f"\nTraining {name}...")
    model = model.to(device)
    optimizer = optim.AdamW(model.parameters(), lr=1e-3, weight_decay=1e-5)

    for epoch in range(EPOCHS):
        train_metrics = train_ner_epoch(
            model, train_loader, optimizer, criterion)
        valid_metrics = evaluate_ner(model, valid_loader, criterion)

        print(f"Epoch {epoch+1}: "
              f"Train F1: {train_metrics['f1']:.3f} | "
              f"Val F1: {valid_metrics['f1']:.3f} | "
              f"Val Acc: {valid_metrics['accuracy']:.3f}")

    print(f"\nFinal Report for {name}:")
    all_preds = []
    all_true = []
    model.eval()
    with torch.no_grad():
        for texts, labels in test_loader:
            texts, labels = texts.to(device), labels.to(device)
            preds = model(texts).argmax(2).view(-1).cpu().numpy()
            targets = labels.view(-1).cpu().numpy()

            mask = targets != -100
            all_preds.extend(preds[mask])
            all_true.extend(targets[mask])

    print(
        classification_report(
            all_true,
            all_preds,
            target_names=[
                id2label[i] for i in range(
                    len(id2label))]))


Training GRU+LSTM...


Training:   0%|          | 0/79 [00:00<?, ?it/s]

Epoch 1: Train F1: 0.372 | Val F1: 0.608 | Val Acc: 0.779


Training:   0%|          | 0/79 [00:00<?, ?it/s]

Epoch 2: Train F1: 0.668 | Val F1: 0.730 | Val Acc: 0.836


...

Epoch 8: Train F1: 0.809 | Val F1: 0.800 | Val Acc: 0.878


Training:   0%|          | 0/79 [00:00<?, ?it/s]

Epoch 9: Train F1: 0.818 | Val F1: 0.807 | Val Acc: 0.881


Training:   0%|          | 0/79 [00:00<?, ?it/s]

Epoch 10: Train F1: 0.827 | Val F1: 0.811 | Val Acc: 0.883

Final Report for CNN + RNN:
              precision    recall  f1-score   support

           O       0.92      0.96      0.94     40499
       B-PER       0.90      0.80      0.85      3543
       I-PER       0.94      0.89      0.91      7544
       B-ORG       0.70      0.63      0.66      4074
       I-ORG       0.75      0.81      0.78      8008
       B-LOC       0.83      0.71      0.76      4560
       I-LOC       0.81      0.75      0.78      3060

    accuracy                           0.88     71288
   macro avg       0.84      0.79      0.81     71288
weighted avg       0.88      0.88      0.88     71288



По метрикам лидирует первая модель (GRU+LSTM). Она же и быстрее всех стартует во время обучения, хотя CNN+RNN в дальнейшем обучается быстрее, выходя на ту же F-меру на валидации раньше. Что касается Residual RNN, у модели, похоже, сложности с обучением, параметры оптимизируются очень медленно, и того же качества она достигает уже в самом конце. Признаков переобучения ни одна модель почти не демонстрирует.

### 8. Функция инференса

In [13]:
import re


def predict_ner(text, model, word2id, id2label):
    model.eval()
    # Простая токенизация
    tokens = re.findall(r'\w+|[^\w\s]+', text)
    ids = [word2id.get(t, 1) for t in tokens]
    tensor_ids = torch.LongTensor([ids]).to(device)

    with torch.no_grad():
        logits = model(tensor_ids)
        preds = logits.argmax(2)[0].cpu().tolist()

    result = []
    for t, p in zip(tokens, preds):
        result.append(f"{t}({id2label[p]})")
    return " ".join(result)

### 9. Тестируем

In [14]:
examples = [
    "Я написал письмо Татьяне Владиславовне, менеджеру ВШЭ.",
    "На Новый год мы в Питере отметили Рождество в баре «Кибер Ель».",
    "Вашингтон объявил новые санкции против Москвы.",
    "Отзывы студентов о питерской Вышке мне не нравятся."
]

for name, model in models_to_train.items():
    print(f"--- Model: {name} ---")
    for ex in examples:
        print(predict_ner(ex, model, word2id, id2label))
    print()

--- Model: GRU+LSTM ---
Я(O) написал(O) письмо(O) Татьяне(O) Владиславовне(I-PER) ,(O) менеджеру(O) ВШЭ(I-PER) .(O)
На(O) Новый(O) год(O) мы(O) в(O) Питере(O) отметили(O) Рождество(O) в(O) баре(O) «(O) Кибер(O) Ель(O) ».(O)
Вашингтон(O) объявил(O) новые(O) санкции(O) против(O) Москвы(B-ORG) .(O)
Отзывы(O) студентов(O) о(O) питерской(O) Вышке(O) мне(O) не(O) нравятся(O) .(O)

--- Model: Residual RNN ---
Я(B-PER) написал(O) письмо(O) Татьяне(O) Владиславовне(O) ,(O) менеджеру(O) ВШЭ(O) .(O)
На(O) Новый(O) год(O) мы(O) в(O) Питере(O) отметили(O) Рождество(O) в(O) баре(O) «(O) Кибер(B-ORG) Ель(I-ORG) ».(I-ORG)
Вашингтон(B-LOC) объявил(I-PER) новые(O) санкции(O) против(O) Москвы(B-LOC) .(O)
Отзывы(B-PER) студентов(O) о(O) питерской(O) Вышке(O) мне(O) не(O) нравятся(O) .(O)

--- Model: CNN + RNN ---
Я(O) написал(O) письмо(O) Татьяне(O) Владиславовне(O) ,(O) менеджеру(O) ВШЭ(O) .(O)
На(O) Новый(B-ORG) год(I-ORG) мы(I-ORG) в(O) Питере(O) отметили(O) Рождество(O) в(O) баре(O) «(O) Кибер(O) Ель(

А вот тут всё очень грустно, все модели отперформили прям плохо. Видимо, потому что предложения максимально не соответствуют тому, что представлено в обучающем датасете — по стилю и названиям (это даже не новости). Но если наши лучшие по метрикам модели (GRU+LSTM и CNN+RNN) скорее консервативные и пропускают названия, то третья (худшая по метрикам) модель наоборот гмассово галлюцинирует их там, где не надо.

В целом, для NER как будто либо правила, либо трансформеры, середина тут так себе работает.